# Tutorial

**Алгоритм применяется для архива аннотаций из бесплатного аккаунта CVAT, в котором аннотации без снимков. Алгоритм добавляет кадры к их аннотациям в соотвествующую по train.txt папку "data/obj_train_data". За тем разбивает массив на выборки.**

Тетрадь для локального запуска, не Google Colab.

Создать папку "data", извлечь в нее файлы архива, этот ноут и видео откуда создавались аннотации в CVAT. Итоговое содержание "data":
 - `create_frames.ipynb` (этот ноут)
 - `obj.data`
 - `obj.names`
 - `obj_train_data`(папка)
 - `train.txt`
 - `video.mp4`**
 
 ** - видео откуда извлекаются кадры, может быть назван как угодно.
 
Далее остается запустить ячейку ниже. Задача выполнена.

Единственный настраиваемый параметр это "annotation_numbers" на случай если нужнен определенный список аннотации. Базово работает повсем.

Примерная скорость работы для снимков 640x640: 1 кадр / 1.45 секунд.

In [ ]:
%%time

import shutil
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
from moviepy.editor import VideoFileClip
from PIL import Image

def folder_cycle(lst: list, root_folder='.'):
    # Путь для создания папки
    for folder_path in lst:
        # Проверка наличия папки
        if not os.path.exists(root_folder+'/'+folder_path):
            # Создание папки, если она не существует
            os.makedirs(root_folder+'/'+folder_path)
            
folder_cycle(['train','valid','test'])
for root_folder in ['train','valid','test']:
    folder_cycle(['images', 'labels'], root_folder)

# функция создания имени кадра. корректное название
correct_number = lambda i: ''.join((6 - len(str(i)))*['0']+[str(i)])

# Путь к видеофайлу
for file_name in os.listdir('.'):
    if file_name.endswith(".mp4"): 
        video_path = file_name
        
# Путь к папке с файлами
folder_path = 'obj_train_data'
# Получение списка файлов в папке
files = os.listdir(folder_path)
# Префикс для добавления
prefix = f"{video_path[:-4]}_"
# Переименование файлов
for file in files:
    if os.path.isfile(os.path.join(folder_path, file)):  # Проверка, что элемент - файл
        new_name = prefix + file
        os.rename(os.path.join(folder_path, file), os.path.join(folder_path, new_name))

# переменная с идентификаторами аннотаций
unique_ids = [int(x[-10:-4]) for x in files if x.endswith(".txt")]

# Разделение на тренировочную (70%) и тестовую+валидационную (30%)
train_data, test_validation_data = train_test_split(unique_ids, test_size=0.3, random_state=42)
# Дальнейшее разделение тестовой+валидационной выборки на тестовую и валидационную (поровну)
validation_data, test_data = train_test_split(test_validation_data, test_size=0.2, random_state=42)

# Загрузка видео
video = VideoFileClip(video_path)

# новое имя файла
new_label = lambda indx: f'{prefix}frame_{correct_number(indx)}'

def change_folder(indx: int, next_folder: str):
    # Имя txt файла и папки исходного места и папки назначения
    file_name = new_label(indx)+'.txt'
    source_folder = folder_path
    destination_folder = next_folder+'/'+'labels'

    # Проверка наличия файла в исходной папке
    if os.path.exists(os.path.join(source_folder, file_name)):
        # Перемещение файла из папки 'test' в папку 'train'
        shutil.move(os.path.join(source_folder, file_name), os.path.join(destination_folder, file_name))

def run(annotation_numbers=None):

    if annotation_numbers == None: 
        annotation_numbers = unique_ids
        
    for i in annotation_numbers:
        # Извлечение конкретного кадра
        frame = video.get_frame(i / video.fps) 

        if i in train_data: new_folder = 'train'
        elif i in validation_data: new_folder = 'valid'
        else: new_folder = 'test'

        # Преобразуем кадр в объект PIL Image
        pil_image = Image.fromarray(frame)

        # Изменяем размер изображения
        new_size = (1280, 1280)
        resized_image = pil_image.resize(new_size)

        # Сохраняем измененный кадр
        resized_image.save(new_folder + '/images/' + new_label(i) + '.jpg')
        
        # перемещаем txt в папку label в соответсвующей выборке
        change_folder(i, new_folder)

    print(f'Задача выполнена. Снимки извлечены. Массив распредлен по выборкам train-valid-test')

# Запуск по всем аннотациям в архиве.
run()


**Тест случайного снимка и его аннотации.**

```
import cv2
import numpy as np
import matplotlib.pyplot as plt

correct_number = lambda i: ''.join((6 - len(str(i)))*['0']+[str(i)])

# Номер кадра
i = 200

# Загрузка изображения
video_label = os.listdir(f'train/images')[0].split('_')[0]
image = cv2.imread(f'train/images/{video_label}_frame_{correct_number(i)}.jpg')
#image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # конвертируем изображение в RGB (matplotlib отображает изображения в формате RGB)
image_rgb = image

# Загрузка данных об ограничивающих рамках из файла frame_000025.txt
with open(f'train/labels/{video_label}_frame_{correct_number(i)}.txt', 'r') as file:
    lines = file.readlines()

# Обработка каждой строки с данными об ограничивающей рамке
for line in lines:
    data = line.strip().split()
    class_id, center_x, center_y, width, height = map(float, data)
    height_image, width_image, _ = image.shape

   # Пересчет координат в координаты углов прямоугольника
    left = int((center_x - width / 2) * width_image)
    top = int((center_y - height / 2) * height_image)
    right = int(left + width * width_image)
    bottom = int(top + height * height_image)

    # Наложение bounding box на изображение
    plt.figure(figsize=(8, 8))
    plt.imshow(image_rgb)
    plt.gca().add_patch(plt.Rectangle((left, top), right - left, bottom - top, linewidth=2, edgecolor='r', facecolor='none'))
    plt.axis('off')  # Отключение осей
    plt.show()
```

**Код создания data.yaml**
```
%%writefile data.yaml

train: ../train/images
val: ../valid/images
test: ../test/images

nc: 10
names: ['Каска', 'Перчатка', 'Обувь', 'Одежда', 'Курение', 'Каска. Нарушение', 'Перчатка. Нарушение', 'Обувью Нарушение', 'Одежда. Нарушение', 'Рабочие']
```